# **Autism Brain disorder prediction using ABIDE dataset**

# Dataset introduction

Autism Brain Imaging Data Exchange (ABIDE) initiative for providing functional and structural brain imaging datasets collected from several brain imaging centers around the world.

Link of ABIDE: https://fcon_1000.projects.nitrc.org/indi/abide/abide_I.html


# Implementation of the project

a.	Apply a linear interpolation method called 'mixup' to enlarge the dataset. \
b.	Apply other AE models as feature extractors: Sparse AutoEncoder(SAE) and Variational AutoEncoder(VAE) \
c.	Construct a DNN model as the final classifier to improve the prediction accuracy.


**The links of papers:**\
mixup: https://arxiv.org/abs/1710.09412 \
SAE: http://minegrado.ovh:3000/General-Team/docs/raw/commit/254ef7b55c2acb53284ec3bc85baa6432760cc4d/ML_DL_NN_books/papers/Autoencoders/sparseAutoencoder_2011new.pdf \
VAE: https://arxiv.org/abs/1312.6114 \

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install matplotlib
!pip install pyprind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from torch.autograd import Variable
import warnings
warnings.filterwarnings('ignore')
from typing import Tuple

## Importing the data 

In [4]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

**Please exchange the path of dataset with your own.**

In [5]:
data_main_path ='/content/drive/MyDrive/DL project/rois_cc200'#cc200'#path to time series data
flist = os.listdir(data_main_path)
print(len(flist))

for f in range(len(flist)):
    flist[f] = get_key(flist[f])

df_labels = pd.read_csv('/content/drive/MyDrive/DL project/Phenotypic_V1_0b_preprocessed1.csv')#path 

df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})
print(len(df_labels))

labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label

1035
1112


### Helper functions for computing correlations

The brain atlas is divided into 200 regions (the raw data of each sample is a time series of those regions). The first thing to do is generated feature vectors, finding those brain regions that highly connected based on the Pearson correlation coefficient. Some helper functions is defined below.

In [53]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]


def get_corr_data(filename):
    #print(filename)
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
            
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(df.corr(method='spearman'))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()

def get_corr_matrix(filename):
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(df.corr(method='spearman'))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0)
    return regions


## Helper functions for computing correlations

The correlation matrix is saved as an pkl file, so that it can be called directly during training to saving time.

In [9]:
p_ROI = "cc200"

In [51]:
if not os.path.exists('./correlations_file'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr = {}
    for f in flist:
      
        lab = get_label(f)
        all_corr[f] = (get_corr_data(f), lab)
        pbar.update()

    print('Corr-computations finished')

    pickle.dump(all_corr, open('./correlations_file'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    all_corr = pickle.load(open('./correlations_file'+p_ROI+'.pkl', 'rb'))

## Computing eigenvalues and eigenvector

The mean of all samples correlation matrix is calculated. Those locations with 1/2 highest score in average correlation matrix is selected and taken as mask, the correlation matrix of each sample is filtered by the mask and flattened to a feature vector.

In [52]:
eig_data = {}
pbar = pyprind.ProgBar(len(flist))
for f in flist:  
      d = get_corr_matrix(f)
      eig_vals, eig_vecs = np.linalg.eig(d)

      for ev in eig_vecs.T:
          np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

      sum_eigvals = np.sum(np.abs(eig_vals))
      # Make a list of (eigenvalue, eigenvector, norm_eigval) tuples
      eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i], np.abs(eig_vals[i])/sum_eigvals)
                    for i in range(len(eig_vals))]

      # Sort the (eigenvalue, eigenvector) tuples from high to low
      eig_pairs.sort(key=lambda x: x[0], reverse=True)

      eig_data[f] = {'eigvals':np.array([ep[0] for ep in eig_pairs]),
                      'norm-eigvals':np.array([ep[2] for ep in eig_pairs]),
                      'eigvecs':[ep[1] for ep in eig_pairs]}
      pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


## Calculating Eros similarity

EROS is a distance metric which is applied to measure the similarity between different samples. The data augmentation is perform between samples and their nearest neighbors.

In [11]:
def norm_weights(sub_flist):
    num_dim = len(eig_data[flist[0]]['eigvals'])
    norm_weights = np.zeros(shape=num_dim)
    for f in sub_flist:
        norm_weights += eig_data[f]['norm-eigvals'] 
    return norm_weights

def cal_similarity(d1, d2, weights, lim=None):
    res = 0.0
    if lim is None:
        weights_arr = weights.copy()
    else:
        weights_arr = weights[:lim].copy()
        weights_arr /= np.sum(weights_arr)
    for i,w in enumerate(weights_arr):
        res += w*np.inner(d1[i], d2[i])
    return res

# Defining Mixup Function(data augmentation)

Mixup is an linear interpolation method, the feature vector of a new sample is generated based on a linear formula. The sample size is doubled after augmentation.

In [12]:
def mixup(x1,x2,y1,y2,alpha,beta):
    """
    get batch data
    :param x: training data
    :param y: one-hot label
    :param step: step
    :param batch_size: batch size
    :param alpha: hyper-parameter α, default as 0.2
    :return:
    """

    if alpha == 0:
        return x,y
    if alpha > 0:
        weight = np.random.beta(alpha, beta, None)
        x_weight = weight
        y_weight = weight
        x = x1 * x_weight + x2 * (1 - x_weight)
        y = y1 * y_weight + y2 * (1 - y_weight)
        return x, y

## Defining dataset class

In [13]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None, 
                 augmentation=False, aug_factor=2, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regs=None):
        self.regs=regs
        if pkl_filename is not None:
            if verbose:
                print ('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        elif data is not None:
            self.data = data.copy()
            
        else:
            sys.stderr.write('Eigther PKL file or data is needed!')
            return 

        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])
        
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]

        
        
        if augmentation:
            self.num_data = aug_factor * len(self.flist)
            self.neighbors = {}
            pbar = pyprind.ProgBar(len(self.flist))
            weights = norm_weights(samples_list)#??
            for f in self.flist:
                label = self.data[f][1]
                candidates = (set(current_lab0_flist) if label == 0 else set(current_lab1_flist))
                candidates.remove(f)
                eig_f = eig_data[f]['eigvecs']
                sim_list = []
                for cand in candidates:
                    eig_cand = eig_data[cand]['eigvecs']
                    sim = similarity_fn(eig_f, eig_cand,weights)
                    sim_list.append((sim, cand))
                sim_list.sort(key=lambda x: x[0], reverse=True)
                self.neighbors[f] = [item[1] for item in sim_list[:num_neighbs]]#list(candidates)#[item[1] for item in sim_list[:num_neighbs]]
        
        else:
            self.num_data = len(self.flist)

        
    def __getitem__(self, index):
        if index < len(self.flist):
            fname = self.flist[index]
            data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode)    
            data = data[self.regs].copy()
            label = (self.labels[index],)
            return torch.FloatTensor(data), torch.FloatTensor(label)
        else:
            f1 = self.flist[index % len(self.flist)]
            d1, y1 = self.data[f1][0], self.data[f1][1]
            d1=d1[self.regs]
            f2=np.random.choice(self.neighbors[f1])
            # f2 = np.random.choice(self.flist)
            d2, y2 = self.data[f2][0], self.data[f2][1]
            d2=d2[self.regs]
            # assert y1 == y2
            # r = np.random.uniform(low=0, high=1)
            # label = (y1,)
            # data = r*d1 + (1-r)*d2
            data,label=mixup(d1,d2,y1,y2,0.2,0.2)
            assert label>=0 and label<=1
            if label>=0 and label<0.5:
              label=0
            elif label>0.5 and label<=1:
              label=1
            else:
              label=np.random.randint(0,2)
            label=(label,)
            return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        return self.num_data

## Definig data loader function

In [14]:
def get_loader(pkl_filename=None, data=None, samples_list=None,
               batch_size=64, 
               num_workers=1, mode='train',
               *, augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False,regions=None):
    """Build and return data loader."""
    if mode == 'train':
        shuffle = True
    else:
        shuffle = False
        augmentation=False

    dataset = CC200Dataset(pkl_filename=pkl_filename, data=data, samples_list=samples_list,
                           augmentation=augmentation, aug_factor=aug_factor, 
                           eig_data=eig_data, similarity_fn=similarity_fn, verbose=verbose,regs=regions)

    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)
  
    return data_loader

In [15]:
# define loss function
def smooth_l1_loss(input, target, sigma=1, reduce=True, normalizer=1.0):
    beta = 1. / (sigma ** 2)
    diff = torch.abs(input - target)
    cond = diff < beta
    loss = torch.where(cond, 0.5 * diff ** 2 / beta, diff - 0.5 * beta)
    if reduce:
        return torch.sum(loss) / normalizer
    return torch.sum(loss, dim=1) / normalizer

# **Perform model training and evaluation(Mixup+VAE+DNN)**

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Define DNN classifier

In [17]:
class DNN(nn.Module):
  def __init__(self):
    super(DNN, self).__init__()
    
    self.fc1 = nn.Linear(9950,4975)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(4975,2000)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(2000,1)

  def forward(self, input_data):
    data = Variable(input_data).to(device)
    x = self.fc1(data) 
    x = self.relu1(x) 
    x = self.fc2(x) 
    x = self.relu2(x)
    pred = self.fc3(x) 
    
    return pred

## Define Variational Autoencoder class



In [18]:
class VAE(nn.Module):
    def __init__(self, DNN,num_inputs=990, 
                 num_latent=200, tied=False,
                 num_classes=2, use_dropout=True):
        super(VAE, self).__init__()
        self.dnn = DNN()
        self.fc1 = nn.Linear(num_inputs, 400)
        self.fc2 = nn.Linear(400, num_latent) 
        self.fc3 = nn.Linear(400, num_latent) 
        self.fc4 = nn.Linear(num_latent, 400)
        self.fc5 = nn.Linear(400, num_inputs)
        if use_dropout:
            self.classifier = nn.Sequential (
                nn.Dropout(p=0.2),
                self.dnn,
                
            )
        else:
            self.classifier = nn.Sequential (
                self.dnn,
            )
    # define endcoder
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    # define reparameter function(random sampling)
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std
 
    # define decoder
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    # define forward training function
    def forward(self, x, eval_classifier = False):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        if eval_classifier:
            x_logit = self.classifier(x)
        else:
            x_logit = None
        return x_reconst, mu, log_var, x_logit
mtae = VAE(DNN)
mtae

VAE(
  (dnn): DNN(
    (fc1): Linear(in_features=9950, out_features=4975, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=4975, out_features=2000, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=2000, out_features=1, bias=True)
  )
  (fc1): Linear(in_features=990, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=400, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=400, bias=True)
  (fc5): Linear(in_features=400, out_features=990, bias=True)
  (classifier): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): DNN(
      (fc1): Linear(in_features=9950, out_features=4975, bias=True)
      (relu1): ReLU()
      (fc2): Linear(in_features=4975, out_features=2000, bias=True)
      (relu2): ReLU()
      (fc3): Linear(in_features=2000, out_features=1, bias=True)
    )
  )
)

## Define training and testing functions



In [19]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()
    train_losses = []
    for i,(batch_x,batch_y) in enumerate(train_loader):
        if len(batch_x) != batch_size:
            continue
        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device)*p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        if mode in ['both', 'ae']:
            if p_bernoulli is not None:
                rec_noisy,mu,log_var,_ = model(data*rand_bernoulli, False)
                kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
                loss_ae = (smooth_l1_loss(rec_noisy, data)+kl_div) / len(batch_x)
            else:
                rec,mu,log_var,_ = model(data, False)
                kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
                loss_ae = (smooth_l1_loss(rec, data)+kl_div) / len(batch_x)
        
        if mode in ['both', 'clf']:
            rec_clean,mu2,var2, logits = model(data, True)
            loss_clf = criterion_clf(logits, target)

        if mode == 'both':
            loss_total = loss_ae + lam_factor*loss_clf
            train_losses.append([loss_ae.detach().cpu().numpy(), 
                                 loss_clf.detach().cpu().numpy()])
        elif mode == 'ae':
            loss_total = loss_ae
            train_losses.append([loss_ae.detach().cpu().numpy(), 
                                 0.0])
        elif mode == 'clf':
            loss_total = loss_clf
            train_losses.append([0.0, 
                                 loss_clf.detach().cpu().numpy()])

        loss_total.backward()
        optimizer.step()

    return train_losses       

def test(model, criterion, test_loader, 
         eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    all_predss=[]
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
        for i,(batch_x,batch_y) in enumerate(test_loader, 1):
            if num_batch is not None:
                if i >= num_batch:
                    continue
            data = batch_x.to(device)
            rec,mu,log_var, logits = model(data, eval_classifier)
            kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            newloss = smooth_l1_loss(rec, data)+kl_div
            test_loss += newloss.detach().cpu().numpy() 
            n_test += len(batch_x)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)
                y_arr = np.array(batch_y, dtype=np.int32)

                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        mlp_acc,mlp_sens,mlp_spef = confusion(y_true,all_predss)

    return  mlp_acc,mlp_sens,mlp_spef#,correct/n_test

# Test the model on whole samples

In [20]:
p_Method = 'Mixup+VAE+DNN'
p_mode = "whole"
p_fold = 10
p_augmentation = True

In [21]:
if p_Method == "Mixup+VAE+DNN" and p_mode == "whole":
    
    num_corr = len(all_corr[flist[0]][0])
    print("num_corr:  ",num_corr)
    
    start =time.time()
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr/4)
    print(n_lat)
    start= time.time()

    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    crossval_res_kol=[]
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)
    kk=0 
    for rp in range(1):
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
        np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
            train_samples, test_samples = flist[train_index], flist[test_index]


            verbose = (True if (kk == 0) else False)

            regions_inds = get_regs(train_samples,int(num_corr/4))

            num_inpp = len(regions_inds)
            n_lat = int(num_inpp/1)
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=eig_data, similarity_fn=sim_function, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size=batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)

            model = VAE(DNN, tied=False, num_inputs=num_inpp,num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)
            
            criterion_clf = nn.BCEWithLogitsLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate_ae)
            #optimizer = optim.SGD([{'params': model.encode.parameters(), 'lr': learning_rate_ae},
                                   #{'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                  #momentum=0.9)

            for epoch in range(10, num_epochs+1):
                if epoch <= 20:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')

      
            res_mlp = test(model, smooth_l1_loss, test_loader, eval_classifier=True)
            print(test(model, smooth_l1_loss, test_loader, eval_classifier=True))
            crossval_res_kol.append(res_mlp)
        print("averages:")
        print(np.mean(np.array(crossval_res_kol),axis = 0))
        finish= time.time()

        print(finish-start)

num_corr:   19900
4975
p_bernoulli:  None
augmentaiton:  True aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 

(0.6923076923076923, 0.5882352941176471, 0.7924528301886793)
(0.7115384615384616, 0.6666666666666666, 0.7547169811320755)
(0.6346153846153846, 0.6274509803921569, 0.6415094339622641)
(0.6826923076923077, 0.5882352941176471, 0.7735849056603774)
(0.7403846153846154, 0.7647058823529411, 0.7169811320754716)
(0.7087378640776699, 0.66, 0.7547169811320755)
(0.6990291262135923, 0.7, 0.6981132075471698)
(0.6796116504854369, 0.58, 0.7735849056603774)
(0.6504854368932039, 0.68, 0.6226415094339622)
(0.6407766990291263, 0.68, 0.6037735849056604)
averages:
[0.68401792 0.65352941 0.71320755]
1566.5823955535889


# Test the model on each site

In [22]:
# set parameter for the model
p_Method = 'Mixup+VAE+DNN'
p_mode="percenter"
p_center="Stanford"
p_fold = 5
p_augmentation = False

In [23]:
if p_Method == "Mixup+VAE+DNN" and p_mode == "percenter":
    num_corr = len(all_corr[flist[0]][0])

    flist = os.listdir(data_main_path)

    for f in range(len(flist)):
        flist[f] = get_key(flist[f])
    
    centers_dict = {}
    for f in flist:
        key = f.split('_')[0]

        if key not in centers_dict:
            centers_dict[key] = []
        centers_dict[key].append(f)

    

    flist = np.array(centers_dict[p_center])
    
    start =time.time()
    #flist = np.array(sorted(os.listdir(data_main_path)))
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr/4)


    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    all_rp_res=[]
    y_arr = np.array([get_label(f) for f in flist])

    kk=0 
    crossval_res_kol_kol=[]
    for rp in range(10):
        print("========================")
        crossval_res_kol = []
        start= time.time()
        kf = StratifiedKFold(n_splits=p_fold)
        #np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
        
            train_samples, test_samples = flist[train_index], flist[test_index]

            verbose = (True if (kk == 0) else False)

            regions_inds = get_regs(train_samples,int(num_corr/4))
            num_inpp = len(regions_inds)
            n_lat = int(num_inpp/1)
            num_inpp = len(regions_inds)
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=eig_data, similarity_fn=sim_function, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size=batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)

            model = VAE(DNN, tied=False, num_inputs=num_inpp,num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)
            criterion_clf = nn.BCEWithLogitsLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate_ae)

            for epoch in range(10, num_epochs+1):
                if epoch <= 20:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')



            res_mlp = test(model, smooth_l1_loss, test_loader, eval_classifier=True)
            #print("fold",kk+1,":",test(model, criterion_ae, test_loader, eval_classifier=True))
            crossval_res_kol.append(res_mlp)
        print("Result of repeat ",rp,":")
        print(np.mean(np.array(crossval_res_kol),axis = 0))
        all_rp_res.append(np.mean(np.array(crossval_res_kol),axis = 0))
        finish= time.time()

        print("Running time:",finish-start)
    print("Avergae result of 10 repeats: ",np.mean(np.array(all_rp_res),axis = 0))

p_bernoulli:  None
augmentaiton:  False aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 

Result of repeat  0 :
[0.69285714 0.58333333 0.8       ]
Running time: 19.491913080215454
Result of repeat  1 :
[0.66785714 0.43333333 0.9       ]
Running time: 19.54474949836731
Result of repeat  2 :
[0.64642857 0.55       0.75      ]
Running time: 21.978393077850342
Result of repeat  3 :
[0.67142857 0.4        0.95      ]
Running time: 19.619850635528564
Result of repeat  4 :
[0.66785714 0.43333333 0.9       ]
Running time: 19.623234510421753
Result of repeat  5 :
[0.74642857 0.6        0.9       ]
Running time: 20.231318712234497
Result of repeat  6 :
[0.58928571 0.46666667 0.7       ]
Running time: 19.655035495758057
Result of repeat  7 :
[0.69642857 0.5        0.9       ]
Running time: 19.766722440719604
Result of repeat  8 :
[0.64642857 0.5        0.8       ]
Running time: 19.69882082939148
Result of repeat  9 :
[0.64285714 0.53333333 0.75      ]
Running time: 19.5501725673675

# **Perform model training and evaluation(Mixup+SAE+DNN)**

# Define DNN classifer for SAE

In [24]:
class DNN(nn.Module):
  def __init__(self):  
    super(DNN, self).__init__()
    
    self.fc1 = nn.Linear(4975,2000)
    self.drop1 = nn.Dropout(p=0.5)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(2000,2)
    self.norm2 = nn.BatchNorm1d(2)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(2,1)

  def forward(self, input_data):
    data = Variable(input_data).to(device)
    x = self.fc1(data)
    x = self.drop1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    x = self.norm2(x)
    x = self.relu2(x)
    pred = self.fc3(x)

    return pred

# Define Sparse Autoencoder class

In [25]:
class AutoEncoderDNN(nn.Module):
    def __init__(self, DNN, num_inputs=990, 
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(AutoEncoderDNN, self).__init__()
        self.tied = tied
        self.num_latent = num_latent
        self.dnn = DNN()
        
        self.fc_encoder = nn.Linear(num_inputs, num_latent)
    
        if not tied:
            self.fc_decoder = nn.Linear(num_latent, num_inputs)
         
        self.fc_encoder = nn.Linear(num_inputs, num_latent)
        
        if use_dropout:
            self.classifier = nn.Sequential (
                nn.Dropout(p=0.5),
                self.dnn,
                
            )
        else:
            self.classifier = nn.Sequential (
                self.dnn,
            )
            
         
    def forward(self, x, eval_classifier=False):
        x = self.fc_encoder(x)
        x = torch.tanh(x)
        if eval_classifier: 
            x_logit = self.classifier(x)
        else:
            x_logit = None
        
        if self.tied: 
            x = F.linear(x, self.fc_encoder.weight.t())
        else:
            x = self.fc_decoder(x)
            
        return x, x_logit 

aednn = AutoEncoderDNN(DNN)

aednn

AutoEncoderDNN(
  (dnn): DNN(
    (fc1): Linear(in_features=4975, out_features=2000, bias=True)
    (drop1): Dropout(p=0.5, inplace=False)
    (relu1): ReLU()
    (fc2): Linear(in_features=2000, out_features=2, bias=True)
    (norm2): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=2, out_features=1, bias=True)
  )
  (fc_encoder): Linear(in_features=990, out_features=200, bias=True)
  (classifier): Sequential(
    (0): DNN(
      (fc1): Linear(in_features=4975, out_features=2000, bias=True)
      (drop1): Dropout(p=0.5, inplace=False)
      (relu1): ReLU()
      (fc2): Linear(in_features=2000, out_features=2, bias=True)
      (norm2): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (fc3): Linear(in_features=2, out_features=1, bias=True)
    )
  )
)

# Define training and testing function

In [43]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()
    train_losses = []
    for i,(batch_x,batch_y) in enumerate(train_loader):
        if len(batch_x) != batch_size:
            continue
        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device)*p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        if mode in ['both', 'ae']:
            if p_bernoulli is not None:
                rec_noisy, _ = model(data*rand_bernoulli, False)
                loss_ae = smooth_l1_loss(rec_noisy, data) / len(batch_x)
            else:
                rec, _ = model(data, False)
                loss_ae = smooth_l1_loss(rec, data) / len(batch_x)

        if mode in ['both', 'clf']:
            rec_clean, logits = model(data, True)
            loss_clf = criterion_clf(logits, target)

        if mode == 'both':
            loss_total = loss_ae + lam_factor*loss_clf
            train_losses.append([loss_ae.detach().cpu().numpy(), 
                                 loss_clf.detach().cpu().numpy()])
        elif mode == 'ae':
            loss_total = loss_ae
            train_losses.append([loss_ae.detach().cpu().numpy(), 
                                 0.0])
        elif mode == 'clf':
            loss_total = loss_clf
            train_losses.append([0.0, 
                                 loss_clf.detach().cpu().numpy()])

        loss_total.backward()
        optimizer.step()

    return train_losses       

def test(model, criterion, test_loader, eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    all_predss=[]
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
        for i,(batch_x,batch_y) in enumerate(test_loader, 1):
            if num_batch is not None:
                if i >= num_batch:
                    continue
            data = batch_x.to(device)
            rec, logits = model(data, eval_classifier) # model(data,true)
            
            test_loss += criterion(rec, data).detach().cpu().numpy() 
            n_test += len(batch_x)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)
                y_arr = np.array(batch_y, dtype=np.int32)

                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        mlp_acc,mlp_sens,mlp_spef = confusion(y_true,all_predss)

    return  mlp_acc,mlp_sens,mlp_spef#,correct/n_test

# Test the model on whole samples

In [41]:
p_Method = 'Mixup+SAE+DNN'
p_mode = "whole"
p_fold = 10
p_augmentation = True

In [49]:
if p_Method == "Mixup+SAE+DNN" and p_mode == "whole":
    
    num_corr = len(all_corr[flist[0]][0])
    print("num_corr:  ",num_corr)
    
    start =time.time()
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr/4)
    print(n_lat)
    start= time.time()

    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    crossval_res_kol=[]
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)
    kk=0 
    for rp in range(1):
        kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
        np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
            train_samples, test_samples = flist[train_index], flist[test_index]


            verbose = (True if (kk == 0) else False)

            regions_inds = get_regs(train_samples,int(num_corr/4))

            num_inpp = len(regions_inds)
            n_lat = int(num_inpp/2)
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=eig_data, similarity_fn=sim_function, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size=batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)

            model = AutoEncoderDNN(DNN,tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)
            
            criterion_clf = nn.BCEWithLogitsLoss()
            criterion_clf.to(device)
            
            optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                   {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                  momentum=0.9)
            '''
            optimizer = optim.Adam([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                   {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                  momentum=0.9)
            '''
            for epoch in range(10, num_epochs+1):
                if epoch <= 20:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')


            res_mlp = test(model, smooth_l1_loss, test_loader, eval_classifier=True)
            print(test(model, smooth_l1_loss, test_loader, eval_classifier=True))
            crossval_res_kol.append(res_mlp)
        print("averages:")
        print(np.mean(np.array(crossval_res_kol),axis = 0))
        finish= time.time()

        print(finish-start)

num_corr:   19900
4975
p_bernoulli:  None
augmentaiton:  True aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 

(0.5, 1.0, 0.0)
(0.5, 0.0, 1.0)
(0.75, 1.0, 0.5)
(0.5, 1.0, 0.0)
(0.5, 0.0, 1.0)
(0.5, 0.0, 1.0)
(0.5, 1.0, 0.0)
(0.5, 0.0, 1.0)
(0.5, 1.0, 0.0)
(0.3333333333333333, 1.0, 0.0)
averages:
[0.50833333 0.6        0.45      ]
62.77608871459961


# Test the model on each site

In [29]:
p_Method="Mixup+SAE+DNN"
p_mode="percenter"
p_center="Stanford"
p_fold=5
p_augmentation = False

In [30]:
if p_Method == "Mixup+SAE+DNN" and p_mode == "percenter":
    num_corr = len(all_corr[flist[0]][0])

    flist = os.listdir(data_main_path)

    for f in range(len(flist)):
        flist[f] = get_key(flist[f])
    
    centers_dict = {}
    for f in flist:
        key = f.split('_')[0]

        if key not in centers_dict:
            centers_dict[key] = []
        centers_dict[key].append(f)

    

    flist = np.array(centers_dict[p_center])
    
    start =time.time()
    #flist = np.array(sorted(os.listdir(data_main_path)))
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr/4)


    print('p_bernoulli: ', p_bernoulli)
    print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')


    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    all_rp_res=[]
    y_arr = np.array([get_label(f) for f in flist])

    kk=0 
    crossval_res_kol_kol=[]
    for rp in range(10):
        print("========================")
        crossval_res_kol = []
        start= time.time()
        kf = StratifiedKFold(n_splits=p_fold)
        #np.random.shuffle(flist)
        y_arr = np.array([get_label(f) for f in flist])
        for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
        
            train_samples, test_samples = flist[train_index], flist[test_index]

            verbose = (True if (kk == 0) else False)

            regions_inds = get_regs(train_samples,int(num_corr/4))
            num_inpp = len(regions_inds)
            n_lat = int(num_inpp/2)
            num_inpp = len(regions_inds)
            train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                    batch_size=batch_size, mode='train',
                                    augmentation=augmentation, aug_factor=aug_factor, 
                                    num_neighbs=num_neighbs, eig_data=eig_data, similarity_fn=sim_function, 
                                    verbose=verbose,regions=regions_inds)

            test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                   batch_size=batch_size, mode='test', augmentation=False, 
                                   verbose=verbose,regions=regions_inds)

            model = AutoEncoderDNN(DNN,tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
            model.to(device)

            criterion_clf = nn.BCEWithLogitsLoss()
            criterion_clf.to(device)
            optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                   {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                   momentum=0.9)

            for epoch in range(10, num_epochs+1):
                if epoch <= 20:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')



            res_mlp = test(model, smooth_l1_loss, test_loader, eval_classifier=True)
            crossval_res_kol.append(res_mlp)
        print("Result of repeat ",rp,":")
        print(np.mean(np.array(crossval_res_kol),axis = 0))
        all_rp_res.append(np.mean(np.array(crossval_res_kol),axis = 0))
        finish= time.time()

        print("Running time:",finish-start)
    print("Avergae result of 10 repeats: ",np.mean(np.array(all_rp_res),axis = 0))

p_bernoulli:  None
augmentaiton:  False aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 

Result of repeat  0 :
[0.48571429 0.85       0.15      ]
Running time: 18.043320655822754
Result of repeat  1 :
[0.43571429 0.5        0.4       ]
Running time: 18.319801568984985
Result of repeat  2 :
[0.53928571 0.25       0.8       ]
Running time: 21.053426504135132
Result of repeat  3 :
[0.46071429 0.5        0.45      ]
Running time: 23.02398657798767
Result of repeat  4 :
[0.48571429 0.4        0.6       ]
Running time: 18.065003633499146
Result of repeat  5 :
[0.51428571 0.2        0.8       ]
Running time: 20.66780686378479
Result of repeat  6 :
[0.48571429 0.65       0.35      ]
Running time: 19.096226453781128
Result of repeat  7 :
[0.51428571 0.         1.        ]
Running time: 19.8189857006073
Result of repeat  8 :
[0.56428571 0.7        0.4       ]
Running time: 18.31317710876465
Result of repeat  9 :
[0.56785714 0.46666667 0.65      ]
Running time: 18.199400663375854


**Evaluation of two model:**\
The prediction accuracy of model 'Mixup+VAE+DNN' is around 69%, while performance of the 'Mixup+VAE+DNN' is relatively poor(around 50%).\
Limited by the sample size, the prediction accuracy of the model seems to be difficult to improve.\
The performance of two model on some sites improved, but perform worse on the others.\
The accuracy of multiple prediction results on one site is also unstable.

**Future direction and further improvement:**
1. find more data
2. develop novel feature extraction method
3. optimize the model (AE and DNN)